In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/p

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [4]:
# What occupation had the highest salary?
df.orderBy(df["salary"].desc()).show(5)

+---+-----------------+---+------------+---------+--------+--------------+---------------+------+------------+
| id|             name|age|height_meter|weight_kg|children|    occupation|academic_degree|salary|    location|
+---+-----------------+---+------------+---------+--------+--------------+---------------+------+------------+
|181|   Jeanice Newman| 51|        1.53|       90|       3|     Operative|            PhD|    90|   Louisiana|
|272|    Sommer Morrow| 54|        1.59|       46|       0|Stage Director|       Bachelor|    90|South Dakota|
|195|     Caprice Kane| 45|        1.72|       61|       4|     Clergyman|         Master|    90|   Louisiana|
| 17|Bernardina Strong| 34|        1.55|       78|       1|     Scientist|            PhD|    90|South Dakota|
|199|     Elli Bernard| 48|        1.56|       77|       5|        Waiter|            PhD|    90|    Delaware|
+---+-----------------+---+------------+---------+--------+--------------+---------------+------+------------+
o

In [5]:
# What occupation had the highest salary?
df.orderBy(df["salary"].asc()).show(5)

+---+---------------+---+------------+---------+--------+-------------------+---------------+------+------------+
| id|           name|age|height_meter|weight_kg|children|         occupation|academic_degree|salary|    location|
+---+---------------+---+------------+---------+--------+-------------------+---------------+------+------------+
|160|   Synthia Tate| 44|        1.77|       77|       0|        Lock Keeper|            PhD|    65|    Delaware|
|257|  Tomoko Carson| 66|        1.92|       43|       1|     Sound Engineer|         Master|    65|    Delaware|
|167|  Catrice Huber| 32|        1.97|       75|       2|      Script Writer|            PhD|    65|    Delaware|
|  7|Melodi Mcdowell| 56|         1.6|       42|       0|  Lighthouse Keeper|         Master|    65|   Louisiana|
|172|      Hae Moody| 39|        1.63|       86|       5|Gaming Club Manager|            PhD|    65|South Dakota|
+---+---------------+---+------------+---------+--------+-------------------+-----------

In [ ]:
# What is the mean salary of this dataset?


In [6]:
# Import average function
from pyspark.sql.functions import avg
df.select(avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     77.738|
+-----------+



In [13]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import min
from pyspark.sql.functions import max 


df.select(min("salary")).show()
df.select(max("salary")).show()

+-----------+
|min(salary)|
+-----------+
|         65|
+-----------+

+-----------+
|max(salary)|
+-----------+
|         90|
+-----------+



In [10]:
# Show all of the occupations where salaries were above 80k
df.filter((df["salary"]>80)).show()

+---+--------------------+---+------------+---------+--------+--------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|          occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+--------------------+---------------+------+-------------+
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|              Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|       Choreographer|       Bachelor|    83|West Virginia|
|  8|Charlesetta Steve...| 30|        1.62|       44|       3|          Millwright|         Master|    87|    Louisiana|
| 11|      Shawnee Harmon| 66|        1.63|       78|       5|   Medical Physicist|            PhD|    90|     Delaware|
| 17|   Bernardina Strong| 34|        1.55|       78|       1|           Scientist|            PhD|    90| South Dakota|
| 21|      Kemberly Yates| 41|  

In [ ]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this

In [16]:
df.groupby (df["academic_degree"])

AttributeError: ignored